In [1]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import time

### Import data

In [2]:
df=pd.read_csv("Eur_USD.csv")

In [3]:
#df.drop(["EURUSD SMA (200D)", "EURUSD SMA (50D)","EURUSD Close"], axis=1, inplace=True)
df.drop("EURUSD Close",axis=1,inplace=True)

In [4]:
df.head()

,Date,EURUSD Open,EURUSD Low,EURUSD High,EURUSD Adj. Close
0,11-17-2020,1.1859,1.1843,1.1894,1.1866
1,11-16-2020,1.1840,1.1806,1.1868,1.1852
2,11-13-2020,1.1806,1.1800,1.1835,1.1833
3,11-12-2020,1.1777,1.1759,1.1822,1.1807
4,11-11-2020,1.1813,1.1747,1.1832,1.1780


In [5]:
df.columns=["Date","Open","Low","High","Adj Close"]

In [6]:
df.head()
df.size

58865

### Technical Indicators

In [7]:
def MACD(DF,fast_MA,slow_MA,macd_signal_period):
    """function to calculate MACD
       typical values: fast_MA = 12; slow_MA =26, macd_signal_period =9
       However, user can enter their own values
       The exponential moving average has been used here to give more weightage to recent prices"""
    df = DF.copy()
    df["MA_Fast"]=df["Adj Close"].ewm(span=fast_MA,min_periods=fast_MA).mean() # fast moving average
    df["MA_Slow"]=df["Adj Close"].ewm(span=slow_MA,min_periods=slow_MA).mean() # slow moving average
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"] # MACD line
    df["Signal"]=df["MACD"].ewm(span=macd_signal_period,min_periods=macd_signal_period).mean() # Signal Line
    #df.dropna(inplace=True)
    df["MACD_Shifted"]=df["MACD"].shift(1)
    df["Signal_Shifted"]=df["Signal"].shift(1)
    #df.dropna(inplace=True)
    
    df.loc[(df["MACD"]>df["Signal"]) & (df["MACD_Shifted"]<df["Signal_Shifted"]),'Trend']='Bullish'
    df.loc[(df["MACD"]<df["Signal"]) & (df["MACD_Shifted"]>df["Signal_Shifted"]),'Trend']='Bearish'
    return df["Trend"]

In [8]:
def ATR(DF,n):
    """function to calculate True Range and Average True Range
       n is the period"""
    df = DF.copy()
    df['High-Low']=abs(df['High']-df['Low']) # Difference between high and low price
    df['High-PreviousClose']=abs(df['High']-df['Adj Close'].shift(1)) # Difference between high and prev_close
    df['Low-PreviousClose']=abs(df['Low']-df['Adj Close'].shift(1)) # Difference between low and prev_close
    df['TR']=df[['High-Low','High-PreviousClose','Low-PreviousClose']].max(axis=1,skipna=False) # Take the maximum of the above 3
    df['ATR'] = df['TR'].rolling(n).mean() # Calculate a rolling moving average based on the number of periods chosen by user
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean() Can also use an exponential moving average
    df2 = df.drop(['High-Low','High-PreviousClose','Low-PreviousClose'],axis=1)
    return df2

In [9]:
def ADX(DF,n):
    """function to calculate ADX
       n is the period
    """
     
    df2 = DF.copy()
    df2['TR'] = ATR(df2,n)['TR'] #the period parameter of ATR function does not matter because period does not influence TR calculation
    df2['DMplus']=np.where((df2['High']-df2['High'].shift(1))>(df2['Low'].shift(1)-df2['Low']),df2['High']-df2['High'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['Low'].shift(1)-df2['Low'])>(df2['High']-df2['High'].shift(1)),df2['Low'].shift(1)-df2['Low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    return df2['ADX']

In [10]:
def slope(points,n):
    "function to calculate the slope of regression line for n consecutive points on a plot"
    slopes = [i*0 for i in range(n-1)]
    for i in range(n,len(points)+1):
        y = points[i-n:i]
        x = np.array(range(n))
        # Scaling to normalize the points 
        y_scaled = (y - y.min())/(y.max() - y.min())
        x_scaled = (x - x.min())/(x.max() - x.min())
        x_scaled = sm.add_constant(x_scaled)
        model = sm.OLS(y_scaled,x_scaled)
        results = model.fit() # Gives the best fit line
        slopes.append(results.params[-1]) # This gives the slope in radians
    slope_angle = (np.rad2deg(np.arctan(np.array(slopes)))) #Convert slope to degrees and then use tan inverse to get numerical value
    return np.array(slope_angle)

### Strategy

In [11]:
df['adx']=ADX(df,20)
df['macd']=MACD(df,25,99,45)

### Dataframe with required indicators 


In [12]:
df

,Date,Open,Low,High,Adj Close,adx,macd
0,11-17-2020,1.1859,1.1843,1.1894,1.1866,NaN,NaN
1,11-16-2020,1.1840,1.1806,1.1868,1.1852,NaN,NaN
2,11-13-2020,1.1806,1.1800,1.1835,1.1833,NaN,NaN
3,11-12-2020,1.1777,1.1759,1.1822,1.1807,NaN,NaN
4,11-11-2020,1.1813,1.1747,1.1832,1.1780,NaN,NaN
...,...,...,...,...,...,...,...
11768,01-08-1975,1.3238,1.3238,1.3238,1.3238,18.341773,NaN
11769,01-07-1975,1.3316,1.3316,1.3316,1.3316,18.043562,NaN
11770,01-06-1975,1.3291,1.3291,1.3291,1.3291,17.881982,NaN
11771,01-03-1975,1.3184,1.3184,1.3184,1.3184,18.202316,NaN


### This strategy generates the following signals:
### 1. If the ADX indicator has a value> 25 and the MACD line was crossing the signal line from below, a Buy signal will be generated
### 2. If the ADX indicator has a value> 25 and the MACD line was crossing the signal line from above, a Sell signal will be generated

In [13]:
df.loc[(df["adx"]>25) & (df['macd']=='Bullish'),'Trade']='Buy'
df.loc[(df["adx"]>25) & (df['macd']=='Bearish'),'Trade']='Sell'
count=0

for data in df['Trade']:
    if data=="Buy" or data=="Sell":
        count+=1
print(count," trades were made in total")


31  trades were made in total


### P&L

In [14]:
trades=[]
price=[]
for data in df["Trade"]:
    if data=="Buy" or data=="Sell":
        trades.append(data)
print(trades)

['Buy', 'Sell', 'Buy', 'Sell', 'Buy', 'Sell', 'Sell', 'Buy', 'Sell', 'Buy', 'Buy', 'Buy', 'Sell', 'Buy', 'Buy', 'Sell', 'Buy', 'Sell', 'Sell', 'Sell', 'Sell', 'Buy', 'Sell', 'Buy', 'Sell', 'Buy', 'Sell', 'Buy', 'Buy', 'Sell', 'Sell']


In [15]:
trading_signal=""
Profit=[]
Buy_price=[]
Sell_price=[]
Quantity=[]
Return=[]
for i in range(len(df)):
    if trading_signal=="":
        Buy_price.append(0)
        Sell_price.append(0)
        Quantity.append(0)
        Profit.append(0)
        Return.append(0)
        if df["Trade"][i]=="Buy":
            trading_signal="Buy"
            Buy_price[i]=(df["Adj Close"][i])
            Quantity[i]=1
        elif df["Trade"][i]=="Sell":
            trading_signal="Sell"
            Sell_price[i]=df["Adj Close"][i]
            Quantity[i]=1
    elif trading_signal=="Buy":
        Buy_price.append(Buy_price[i-1])
        Quantity.append(Quantity[i-1])
        Sell_price.append(Sell_price[i-1])
        Profit.append(0)
        Return.append(0)
        if df["Trade"][i]=="Buy":
            Buy_price[i]+=df["Adj Close"][i]
            Quantity[i]+=1
        elif df["Trade"][i]=="Sell":
            Sell_price[i]=df["Adj Close"][i]
            Profit[i]=(Sell_price[i]*Quantity[i])-Buy_price[i]
            Return[i]=(((Sell_price[i]*Quantity[i])-Buy_price[i])/Buy_price[i])
            trading_signal=""
    elif trading_signal=="Sell":
        Sell_price.append(Sell_price[i-1])
        Quantity.append(Quantity[i-1])
        Buy_price.append(Buy_price[i-1])
        Profit.append(0)
        Return.append(0)
        if df["Trade"][i]=="Sell":
            Sell_price[i]+=df["Adj Close"][i]
            Quantity[i]+=1
        elif df["Trade"][i]=="Buy":
            Buy_price[i]=df["Adj Close"][i]
            Profit[i]=Sell_price[i]-(Buy_price[i]*Quantity[i])
            Return[i]=((Sell_price[i]-(Buy_price[i]*Quantity[i]))/(Buy_price[i]*Quantity[i]))
            trading_signal=""   

In [16]:
df["Buy_price"]=Buy_price
df["Sell_price"]=Sell_price
df["Quantity"]=Quantity
df["Profit"]=Profit
df["Return"]=Return

In [17]:
df

,Date,Open,Low,High,Adj Close,adx,macd,Trade,Buy_price,Sell_price,Quantity,Profit,Return
0,11-17-2020,1.1859,1.1843,1.1894,1.1866,NaN,NaN,NaN,0.0,0.0000,0,0.0,0.0
1,11-16-2020,1.1840,1.1806,1.1868,1.1852,NaN,NaN,NaN,0.0,0.0000,0,0.0,0.0
2,11-13-2020,1.1806,1.1800,1.1835,1.1833,NaN,NaN,NaN,0.0,0.0000,0,0.0,0.0
3,11-12-2020,1.1777,1.1759,1.1822,1.1807,NaN,NaN,NaN,0.0,0.0000,0,0.0,0.0
4,11-11-2020,1.1813,1.1747,1.1832,1.1780,NaN,NaN,NaN,0.0,0.0000,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11768,01-08-1975,1.3238,1.3238,1.3238,1.3238,18.341773,NaN,NaN,0.0,1.3641,1,0.0,0.0
11769,01-07-1975,1.3316,1.3316,1.3316,1.3316,18.043562,NaN,NaN,0.0,1.3641,1,0.0,0.0
11770,01-06-1975,1.3291,1.3291,1.3291,1.3291,17.881982,NaN,NaN,0.0,1.3641,1,0.0,0.0
11771,01-03-1975,1.3184,1.3184,1.3184,1.3184,18.202316,NaN,NaN,0.0,1.3641,1,0.0,0.0


In [18]:
def CAGR(DF,x):
    """function to calculate the Cumulative Annual Growth Rate of a trading strategy
       x is the number of times the strategy runs in a day"""
    df = DF.copy()
    df["cum_return"] = (1 + df["Return"]).cumprod()
    n = len(df)/(252*x) 
    CAGR = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

In [19]:
def volatility(DF,x):
    """function to calculate annualized volatility of a trading strategy
       x is the number of times the strategy will run in a day"""
    df = DF.copy()
    vol = df["Return"].std() * np.sqrt(252*x)
    return vol


In [20]:
def sharpe(DF,rf,x):
    """function to calculate sharpe ratio
      rf is the risk free rate
      x is the number of times the strategy will run in a day"""
    df = DF.copy()
    sr = (CAGR(df,x) - rf)/volatility(df,x)
    return sr

In [21]:
sharpe(df,0,1)

0.22658732065332549

In [22]:
CAGR(df,1)

0.02167320687415364

In [23]:
volatility(df,1)

0.09565057220175728